# Data Ingestion
This notebook demonstrates the process of ingesting raw data into the bronze layer using PySpark.


In [6]:
from pyspark.sql import SparkSession
from datetime import datetime
import os
import logging

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Data Ingestion") \
    .getOrCreate()

In [11]:
# Define paths
base_path = "../data/raw/"
bronze_base_path = "../output/bronze/"
date_str = datetime.now().strftime("%Y-%m-%d")

# Log path organized by date
log_dir = os.path.join("logs", "data_ingestion", date_str)
log_path = os.path.join(log_dir, "data_ingestion.log")

# Ensure the logs directory exists
os.makedirs(log_dir, exist_ok=True)
print(log_dir+"\n"+ log_path)

logs\data_ingestion\2024-11-04
logs\data_ingestion\2024-11-04\data_ingestion.log


In [ ]:
# Configure logging to write to the log file
logging.basicConfig(filename=log_path, level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def log_message(message, level="info"):
    """Logs messages with the specified level."""
    if level == "info":
        logging.info(message)
    elif level == "error":
        logging.error(message)
    print(message)  

In [14]:
# Define sources
sources = ["sales", "customers", "products", "suppliers"]
print(sources)

['sales', 'customers', 'products', 'suppliers']


In [15]:
# Process each source file
for source in sources:
    try:
        raw_path = os.path.join(base_path, f"{source}.csv")
        bronze_path = os.path.join(bronze_base_path, source, date_str)

        # Log start of the ingestion process for this source
        log_message(f"Starting ingestion for {source} from {raw_path} to {bronze_path}")

        # Read raw data
        df = spark.read.csv(raw_path, header=True, inferSchema=True)
        
        # Ensure the bronze path exists
        os.makedirs(bronze_path, exist_ok=True)
        
        # Write to Bronze layer in Parquet format, organized by date
        df.write.mode("overwrite").parquet(bronze_path)
        
        # Log successful ingestion
        log_message(f"Successfully ingested {source} data to {bronze_path}")
        
    except Exception as e:
        # Log any errors encountered during ingestion
        log_message(f"Error ingesting {source} data: {e}", level="error")

Starting ingestion for sales from ../data/raw/sales.csv to ../output/bronze/sales\2024-11-04
Successfully ingested sales data to ../output/bronze/sales\2024-11-04
Starting ingestion for customers from ../data/raw/customers.csv to ../output/bronze/customers\2024-11-04
Successfully ingested customers data to ../output/bronze/customers\2024-11-04
Starting ingestion for products from ../data/raw/products.csv to ../output/bronze/products\2024-11-04
Successfully ingested products data to ../output/bronze/products\2024-11-04
Starting ingestion for suppliers from ../data/raw/suppliers.csv to ../output/bronze/suppliers\2024-11-04
Successfully ingested suppliers data to ../output/bronze/suppliers\2024-11-04


In [16]:
# Stop Spark session
spark.stop()
log_message("Data ingestion process completed.")

Data ingestion process completed.
